# BAN438 EXAM
Date: 22.05.2023 \
Candidate Number: 72 

Code which has been inspired by other authors than myself will be referenced in the text. I will not reference the Plotly and Dash documentation individually, as I have taken much inspiration from the features presented there. The inspiration behind the various code sections and development process has come from the lectures, workshop and other resources from the BAN438 Canvas page.

As a creative choice, I have decided not to add any text which is not present in the original dashboard. Even though the dashboard could use some explanation as to the functionality. This is because the reference dashboard has chosen not to include pointers and guides as to how to use their dashboard, which made it easier for me to choose a similar approach. All the functionality of the dashboard will mainly be explained via in-line comments, whilst the functionality of the dashboard itself "speaks for itself".

In [2]:
# These packages will be used in the project
import pandas as pd                                          # Reading and Organizing Data
import dash                                                  # Running the Application
from dash import dcc                                         # Creating Interactive Components
from dash import html                                        # Allow for the use of HTML tags
import plotly.express as px                                  # Creating Interactive Plots
from dash.dependencies import Input, Output                  # Neccesary for interactivity in the app
import dash_bootstrap_components as dbc                      # Used for styling of application
from dash_bootstrap_templates import load_figure_template    # Used for styling of
from jupyter_dash import JupyterDash                         # Jupyter Notebook dash integration
import datetime as datetime                                  # Time Utilities
from dateutil.relativedelta import relativedelta             # Time Utilities

# Import Style Files
# I have chosen the "Litera" template from bootstrap as the visual
# foundation of this dashboard
dbc_css = 'https://bootswatch.com/5/litera/bootstrap.min.css'
load_figure_template('litera')

# Inn addition to this, I have created some modifications to this template in my own custom
# styles.css file located in \assets\styles.css.

In [3]:
# :-------------------------------
# Loading in the data set used as a basis of this dashboard

# Use the URL which the .csv file is located to download and read the data
url = "https://data.norges-bank.no/api/data/EXR/B..NOK.SP?startPeriod=2013-05-22&endPeriod=2023-05-22&format=csv&bom=include&locale=en"
forex = pd.read_csv(url, sep=";", low_memory=False)

# If this data does not work, I have included the EXR.csv file which I used in
# creating the dashboard in the "data" folder in this zip file. Although is should
# not be neccecary as I have created the dashboard with futureproofing in mind.

# forex = pd.read_csv("data/EXR.csv", sep=";", low_memory=False)


# Useful features for this task
exam_columns = ["BASE_CUR", "Base Currency", "Unit Multiplier", "TIME_PERIOD", "OBS_VALUE"]
forex = forex[exam_columns]

# With the data imported we can make use of the .head() and .info() functions to
# get some insight into the dataset at hand
forex.head()

,BASE_CUR,Base Currency,Unit Multiplier,TIME_PERIOD,OBS_VALUE
0,HUF,Hungarian forint,Hundreds,2013-05-22,2.5825
1,HUF,Hungarian forint,Hundreds,2013-05-23,2.5785
2,HUF,Hungarian forint,Hundreds,2013-05-24,2.5925
3,HUF,Hungarian forint,Hundreds,2013-05-27,2.6097
4,HUF,Hungarian forint,Hundreds,2013-05-28,2.6423


In [4]:
# Getting a look into the data types and number of observations
forex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96872 entries, 0 to 96871
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BASE_CUR         96872 non-null  object
 1   Base Currency    96872 non-null  object
 2   Unit Multiplier  96872 non-null  object
 3   TIME_PERIOD      96872 non-null  object
 4   OBS_VALUE        96872 non-null  object
dtypes: object(5)
memory usage: 3.7+ MB


In [5]:
# From this it is obvious that we have to reprogram some of the columns to the correct Dtype
# The TIME_PERIOD has to be converted to a datetime Dtype
forex['TIME_PERIOD'] = pd.to_datetime(forex['TIME_PERIOD'])

# The OBS_VALUE is considered a string, sp we have to convert it to numeric
# I use the coerce error handling in order to find any non numeric values in
# the data frame
forex['OBS_VALUE'] = pd.to_numeric(forex['OBS_VALUE'], errors='coerce')

# Therefore, it is important that we find what caused the NA values to appear
forex.isna().sum(axis=0)

BASE_CUR             0
Base Currency        0
Unit Multiplier      0
TIME_PERIOD          0
OBS_VALUE          349
dtype: int64

In [6]:
# Because of our converting, 349 NA values appeared in the OBS_VALUE dataset.
# Although this is not preferable, I deem the amount small enough to just filter them out and 
forex = forex.dropna(subset=['OBS_VALUE'])

In [7]:
# Because of the nature of these tasks, the data which has been added to the list most recently
# needs to be accesed quite often. As the data is published around 16:00 each working day, I will
# use the .max() function in order to determine which day is the latest day with information about
# the currencies
today = forex["TIME_PERIOD"].max()
first_day = forex["TIME_PERIOD"].min()

In [8]:
# In order to check that Norges Bank has the latest conversion rates for each currency,
# I display all the maximum time periods in each category
currency_date_df = forex.groupby('BASE_CUR').agg({'TIME_PERIOD': 'max'}).reset_index()
currency_date_miss = currency_date_df[currency_date_df["TIME_PERIOD"] != today]
currency_date_miss

,BASE_CUR,TIME_PERIOD
6,CHF,2022-06-15
13,HRK,2022-12-30
30,RUB,2022-03-01


In [9]:
# Here we see that three currencies are not up to date.
# We therefore need to make sure that these are know later in the tasks

# The reasoning behind their exclusion is as follow:
# - Russia is barred from the international monetary system, and rubles therefore has
#   no conversion value
# - Croatia switched from the kuna to Euro this year, which explain the lack of information
# - The Swiss Franc conversion rate error is hard to explain

warning_currency = currency_date_miss["BASE_CUR"].tolist()
warning_currency
# The data gathered from the Swiss franc is not very good, as the last observation in my data set
# is 15.06.2022, even though the online dashboard has many more observations.
# I will try to implement support for Swiss Franc as best as I can, although it is not easy to work around

['CHF', 'HRK', 'RUB']

With most of the data wrangled and cleaned, we can start implementing some of the features given in the task description. Later in the task we will make some additional transformations, although this is not relevant for all the features of the task.

## Developing the features

In [57]:
# :-------------------------------
# Header Dashboard
# :-------------------------------
# Here we make use of a custom CSS file in order to
# resize and color the background picture

header_card = dbc.Card(
    [
        dbc.CardImg(
            src="/assets/bergen_drone.JPG",     # Reference the drone picture stored in the assets folder
            top=True,                           # The following lines of code are mainly aesthetic changes
            className="card-image",
            style={"opacity": 0.3}
        ),
        dbc.CardImgOverlay(
            dbc.CardBody(
                [
                    dbc.Row(children=[
                        dbc.Col(html.H4('BAN438 | CANDIDATE 72', style = {'textAlign' : 'center', 'color':'white'}))
                         ]),
                    
                    dbc.Row(children=[
                        dbc.Col(html.H1('Exchange Rate Dashboard', style = {'textAlign' : 'center', "fontSize":"300%", 'color':'white'}))
                        ]),
                ],
            ),
            className="overlay",
        ),
    ],
    style={"width": "12"},
)

# :-------------------------------
# Body Text of the Dashboard
# I write a simple description of the dashboard as a whole 
# in html, and store it as the body text of a dbc.Card

description_dash = dbc.Card([html.Span("Norges Bank quotes some 40 exchange rates. Publication time of daily exchange rates is approximately 16:00 CET. "),
                            html.Span("The link to the original dashboard can be found "),
                            html.A('here', href='https://www.norges-bank.no/en/topics/Statistics/exchange_rates/'),
                            html.Span(". The data can be downloaded from "),
                            html.A('this', href=url),
                            html.Span(" link.")], body = True)
 

body_card = html.Div(
    [
        description_dash,
    ]
)

In [58]:
# Intro Modal
# :-------------------------------
description_modal = """
This exam has been submitted by Candidate 72, and aims to produce a dashboard of various conversion rates between foreign currencies
and Norwegian Kroner (NOK). The dashboard was made the 22.05.2023 and has been designed with the data availbale data in mind. 
However, there should not be any compatability errors as there are no direct refrences to tickers, dates and rates within the
code itself. Enjoy the dashboard!
"""

intro_modal = dbc.Modal(
            [
                dbc.ModalHeader(dbc.ModalTitle("BAN438 EXAM SPRING 2023")),
                dbc.ModalBody(dcc.Markdown(description_modal)),
            ],
            id="modal-lg",
            size="lg",
            is_open=True,
            centered=True
            )


In [59]:
# :-------------------------------
# Feature 1 Data Formatting
# :-------------------------------

# This feature will display a static data table which updates based upon the last two days with observations
# for the echange rate between NOK and other currencies

# Copy the concents of forex to a df1 for feature 1
df1 = forex.copy()

# First we need to etablish the last two days with data
# We do this by removing the max feature 'today', and finding the new max
yesterday = df1[df1["TIME_PERIOD"] != today]
yesterday = yesterday["TIME_PERIOD"].max()

# Then we can filter out values which do not contain eiter of these two values
df1 = df1[df1["TIME_PERIOD"].isin([today, yesterday])]

# Then we convert it from a long format to a wider format data frame
pivot_df1 = df1.pivot_table(index=['BASE_CUR', 'Base Currency', 'Unit Multiplier'], columns='TIME_PERIOD', values='OBS_VALUE')

# Reset the index
pivot_df1.reset_index(inplace=True)

# Rename Date axis
column_names = pivot_df1.columns.tolist()
yesterday_date = column_names[3]
today_date = column_names[4]

# Converting and storing the date string back into the column names
formatted_date_y = yesterday_date.strftime('%d %b')
column_names[3] = formatted_date_y

formatted_date_t = today_date.strftime('%d %b')
column_names[4] = formatted_date_t

# Storing the changed column names back into the data frame
pivot_df1.columns = column_names

# Rename columns
pivot_df1.rename(columns={'BASE_CUR': 'Code',
                          'Base Currency':"Currency",
                          'Unit Multiplier': 'Multiplier'}, inplace=True)

# Remove the values which is in the warning list
pivot_df1 = pivot_df1[~pivot_df1["Code"].isin(warning_currency)]

# We can look at the "finished" data set using the .head() function
pivot_df1.head()

# Note! Although the official dashboard did not include the multiplier, I found it somewhat
# confusing. I have therefore chosen to include it in my dashboard!

,Code,Currency,Multiplier,16 May,19 May
0,AUD,Australian dollar,Units,7.1215,7.2090
1,BDT,Bangladeshi taka,Hundreds,9.9400,10.0900
2,BGN,Bulgarian lev,Hundreds,593.7700,598.2700
3,BRL,Brazilian real,Units,2.1772,2.1849
4,BYN,Belarusian new rouble,Units,4.2283,4.2891


In [60]:
# :-------------------------------
# Feature 2 Data Formatting
# :-------------------------------

# We use the raw "forex" data in order to get an accuracte value
# for conversion rate. Therefore we dont need to do any data wrangling
# before the callback function defined later in the task.

In [61]:
# :-------------------------------
# Feature 3 Data Formatting
# :-------------------------------

# Then we can make the graphs. This will be split into three seperate data sets
# based on the time interval chosen by the user

# We define a function which aggregates based on specified timeframe
# in order to avoid code repetition
def aggegate_currency(time_period):
    df_agg = forex.copy()
    
    # Create a new column for the start of timeframe
    df_agg['agg_start'] = df_agg['TIME_PERIOD'].dt.to_period(time_period).apply(lambda r: r.start_time)
    
    # Now group by 'BASE_CUR' and 'timeperiod_start', and calculate the mean of 'OBS_VALUE'
    df_agg = df_agg.groupby(['BASE_CUR', 'agg_start'])['OBS_VALUE'].mean().reset_index()
    
    # Rename the 'OBS_VALUE'
    df_agg.rename(columns={'OBS_VALUE': 'agg_avg_obs', 'BASE_CUR':'ticker'}, inplace=True)
    return(df_agg)
    

# :-------------------------------
# For the "Day" tab
df_day = aggegate_currency("D")     # Not neccesary, but I get the same formatting by inserting it into the function

# :-------------------------------
# For the "Month" tab
df_month = aggegate_currency("M")

# :-------------------------------
# For the "Year" tab
df_year = aggegate_currency("Y")

In [62]:
# :-------------------------------
# User Input
# :-------------------------------

# In this code section, the various ways that a user can interact
# with the dashboard will be established

# :-------------------------------
# Number of units
number_input = dbc.Card(                        # This input is inspired by the example solution from Candidate 6 in         
    [                                           # last years exam (both this part and the callback)
        dbc.Input(type="number", id="numeric-input", value = 1),
    ],
)

# :-------------------------------
# Currency Dropdown

# In order to get the naming convention right, I have created a lambda function
# which ensures that the data from the BASE_CUR is listed besides the full name
# in parenthesis
currency_options = forex.drop_duplicates(subset=['BASE_CUR', 'Base Currency']).apply(
    lambda row: {"label": f"{row['Base Currency']} ({row['BASE_CUR']})", "value": row['BASE_CUR']},
    axis=1).tolist()

currency_dropdown = dcc.Dropdown(
    id = "currency-dropdown",
    options=currency_options,
    value = "USD",
    multi=False,
    clearable=False
)

# :-------------------------------
# Date Input Start
date_input_start = dcc.DatePickerSingle(
        id='date-start',
        min_date_allowed=first_day,
        max_date_allowed=today,
        initial_visible_month=first_day,
        date=first_day
    ),

# :-------------------------------
# Date Input End
date_input_end = dcc.DatePickerSingle(
        id='date-end',
        min_date_allowed=first_day,
        max_date_allowed=today,
        initial_visible_month=today,
        date=today
    ),

# :-------------------------------
# Time frame radio buttons
timeframe_radio = html.Div(
    [
        dbc.Label(html.H4("Select Timeframe:")),
        dbc.RadioItems(
            options = {"day":"Day",
                        "month":"Month",
                        "year":"Year"},
            value="day",
            id="radioitems-inline-input",
            inline=True,
        ),
    ]
)

# :-------------------------------
# Reset Time Index
reset_time = dbc.Button("Reset Time", id='reset-button', className="me-2")

In [63]:
# Now the cards can be created

# :-------------------------------
# Feature 1
table_description = "Norges Bank's exchange rates are middle rates, i.e. the mid-point between buying and selling rates in the interbank market at a given time. The exchange rates are only intended to serve as an indication, and are not binding on Norges Bank or other banks."

card_table = dbc.Card(children=[
    dbc.Row(dbc.Col(dbc.CardHeader(html.B('Feature 1:'), style = {"fontSize":"200%"}, className="standard-font"))),
    dbc.Row(dbc.Col(dbc.CardBody(table_description))),
    
    # html.Br(), 
    html.Br(),
    
    # Table
    dbc.Row(children=[
        dbc.Col(
            dbc.Table.from_dataframe(pivot_df1, striped = True, bordered = True, hover = True)
        )
    ], justify="centre"),

], style = {"width" : "12"})

# :-------------------------------
# Feature 2
card_currency = dbc.Card(children=[
    # A header displaying that this top part is Feature 2
    dbc.Row(dbc.Col(dbc.CardHeader(html.B('Feature 2:'), style = {"fontSize":"200%"}, className="standard-font"))),
    
    # This part contains the layout of both Feature 2 and 3
    dbc.Container(children=[
        html.Br(),
        dbc.Row(children=[
            dbc.Col(number_input, width=3, className="mx-auto text-center"),                      # These columns represent the currency converter inputs,
                                                                                                  # which currency is picked and the final conversions
            dbc.Col(children = [currency_dropdown, html.Br(),
                                dbc.Row(children=[                                                # This is the feature which shows which day the last
                                    dbc.Col(html.B(id="exchange-metric"), width = 6),             # price update is gathered from, as we have some currencies
                                    dbc.Col(html.P(id = "currency-calc-info"), width = 6)         # which has not been updated for a while, and which measurement unit is used (1 or 100)
                                ])
                                ], width=4, className="text-center"),
            
            dbc.Col(html.H4(id="currency-output"), width=3, className="mx-auto text-left")
        ], justify="center"),
    ]),
    html.Br(),
    dbc.Card(children=[
        dbc.Row(dbc.Col(dbc.CardHeader(html.B('Feature 3:'), style = {"fontSize":"200%"}, className="standard-font"))),
        html.Br(),
        dbc.Row(dbc.Col(timeframe_radio, width=3, className="mx-auto text-center")),
        html.Br(),
        
        # This section contains the various metrics for choosing which time frame to display the line graph in Feature 3 in.
        # It allows both for the use of date pickers, and a button which resets the time to the earliest and newest observation
        # in the downloaded data set
        
        dbc.Row(children=[
            dbc.Col(html.H4("Show Period:"), width=2, className="d-flex align-items-center justify-content-center"),
            dbc.Col(width=1),
            dbc.Col(html.B("from:"), width=1, className="d-flex align-items-center justify-content-center"),
            dbc.Col(date_input_start, width= 1),
            dbc.Col(width=1),
            dbc.Col(html.B("to:"), width=1, className="d-flex align-items-center justify-content-center"),
            dbc.Col(date_input_end, width= 2),
            dbc.Col(width=1),
            dbc.Col(reset_time)
            
        ]),

        # The task description does not excplicitly ask for the table which is shown to the left of the line
        # diagram in the reference dashboard, so I will not include it. Although it would be a cool addition
        # given more time to implement.

        html.Br(),
        dbc.Row(dbc.Col([dcc.Graph(id="line-forex")])),
        html.Br()
        
    ])
])


In [64]:
# Defining the two main data tabs in the dashboard
tab1 = dbc.Tab(children = [card_table], label = "All Exchange Rates")

tab2 = dbc.Tab(children = [card_currency], label = "Choose Currency")

In [65]:
# :-------------------------------
# Creating a dash framework
load_figure_template('litera')
app = JupyterDash(external_stylesheets = [dbc.themes.LITERA, dbc_css])

app.layout = dbc.Container(
    children = [

        # Intro Modal (can be disabled if it becomes annoying
        intro_modal,
        
        # Placing the Header and the Body card
        # In the layout
        html.Br(),
        header_card,
        html.Br(),
        body_card,
        html.Br(),

        # Add in both the table chart tab and
        # the line chart tab
        dbc.Tabs(children = [
            tab1, 
            tab2
        ]), 
        html.Br()
    ]
)

# :-------------------------------
# Callbacks
# :-------------------------------

# :-------------------------------
# Currency Calculator
@app.callback(
    Output("currency-output", "children"),
    Output("currency-calc-info", "children"),
    Input("currency-dropdown", "value"),
    Input("numeric-input", "value")
)
def convert(ticker, value_input):
    if value_input == None:
        return("= 0.00 NOK", "No Input Detected")
    
    # Define a list of possible multipliers
    multiplierlist = forex["Unit Multiplier"].unique().tolist()
    
    if ticker in warning_currency:
       last_date = currency_date_miss[currency_date_miss["BASE_CUR"] == ticker]
       last_date = last_date["TIME_PERIOD"].item()
       
       # First I extract all values from the today value, to get the most accurate 
       # reading of the conversion rate
       rate = forex[forex["TIME_PERIOD"] == last_date].reset_index(drop = True)
       
       # Defining the date used
       date_used = last_date
    
    else:
        # First I extract all values from the today value, to get the most accurate
        # reading of the conversion rate
        rate = forex[forex["TIME_PERIOD"] == today].reset_index(drop = True)
        
        # Defining the date used
        date_used = today

    # Then we can extract the data from the relevant ticker
    rate = rate[rate["BASE_CUR"] == ticker]
    
    # The we need to define wheter or not the OBS_VALUE
    # is listed in Hundreds or Units
    if rate["Unit Multiplier"].item() == multiplierlist[1]:
        currency_converted = rate["OBS_VALUE"].item() * value_input
    elif rate["Unit Multiplier"].item() == multiplierlist[0]:
        currency_converted = (rate["OBS_VALUE"].item())/(100)* value_input    
    
    # We return both the output of the converter, and the last time the rate updated
    return (f"= {round(currency_converted, 2)} NOK", 
            html.Span([f"Updated {date_used.strftime('%d/%m/%Y')}"], style={'font-style': 'italic'}))

# :-------------------------------
# Graph Callback
@app.callback(
    Output("line-forex", "figure"),
    Input("currency-dropdown", "value"),
    Input("radioitems-inline-input", "value"),
    Input("date-start", "date"),
    Input("date-end", "date")
)
def plot_line_graph(ticker, timeframe, start, end):
    # If invalid Options are inserted, it defaults to 
    # the first possible and last possible day
    if start == None:
        start = first_day
    
    if end == None:
        end = today
        
    
    # Check which timeframe is chosen, and storing in the subset
    # ! One addiational feature is customer hover info, depending on which
    #   time frame is chosen. This is done by adding the 'date_str' column
    if timeframe == "day":
        subset = df_day.copy()
        subset['date_str'] = subset['agg_start'].dt.strftime('%d.%m.%Y')
    elif timeframe == "month":
        subset = df_month.copy()
        subset['date_str'] = subset['agg_start'].dt.strftime('%B %Y')
    elif timeframe == "year":
        subset = df_year.copy()
        subset['date_str'] = subset['agg_start'].dt.strftime('%Y')

    # One small "feature": I dont think it is good practise to compare averages of 
    # incomplete years to complete years, therefore I have chosen to not include incomplete
    # years (such as 2013 at the time of making this) in the years tab as it is not comparable 
    # to averages over a whole year. It will still be visible in the day and month section though.
    
    # Filtering out by ticker
    subset = subset[subset["ticker"] == ticker]
    
    # Filtering out by specified start/end date
    subset = subset[(subset["agg_start"] >= start) & (subset["agg_start"] <= end)]
    
    # Defining the Hover label
    subset['hover_label'] = '<b>' + subset['ticker'] + '-NOK' + "</b>" + '<br>' + subset['agg_avg_obs'].round(3).astype(str) + ' NOK' + '<br>' + subset['date_str']
    
    # Creating the plot
    fig = px.line(
        data_frame=subset,
        x="agg_start",
        y="agg_avg_obs",
        color="ticker",
        labels={"ticker":"",
                "agg_avg_obs":"Average Conversion Rate",
                "agg_start":""},
        title=f"Line Plot of {ticker} against NOK per {timeframe}",
        hover_data=['hover_label']
    )
    
    # Adding the hover label
    fig.update_traces(hovertemplate='%{customdata[0]}')

    # Returning the figure to the Graph object
    return(fig)
        

# :-------------------------------
# Not allowing date overlap by setting the current start/end
# dates as thresholds for each other
@app.callback(
    Output("date-end", "min_date_allowed"),
    Output("date-start", "max_date_allowed"),
    Input("date-end", "date"),
    Input("date-start", "date")
)
def max_dates(current_end, current_start):
    if current_end != None:
        return(current_start, current_end)
    else:
        return(first_day, today)
    
# :-------------------------------
# Finding the exchange metric used in the conversion
@app.callback(
    Output("exchange-metric", "children"),
    Output("numeric-input", "value"),
    Input("currency-dropdown", "value")
)
def exchange_metric(ticker):
    new_df = forex[['BASE_CUR', 'Unit Multiplier']].drop_duplicates().reset_index(drop=True)
    
    # Find the unit used for a given ticker
    ticker_unit = new_df[new_df["BASE_CUR"] == ticker]
    ticker_unit = ticker_unit["Unit Multiplier"].item()

    # Displaying based on which unit is chosen
    if ticker_unit == "Units":
        return(f"NOK PR. 1 {ticker}", 1)
    elif ticker_unit == "Hundreds":
        return(f"NOK PR. 100 {ticker}", 100)
    
# :-------------------------------
# Reset Time Button
@app.callback(
    Output('date-start', 'date'),
    Output('date-end', 'date'),
    Input('reset-button', 'n_clicks')
)
def update_dropdown(clicks):
    if clicks is None:  # No button click
        raise dash.exceptions.PreventUpdate
    
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'reset-button':
        return (first_day, today)
    
    raise dash.exceptions.PreventUpdate


# :-------------------------------



# :-------------------------------
# Run the App
app.run_server(debug = True, port = 8052)

Dash is running on http://127.0.0.1:8052/

Dash app running on http://127.0.0.1:8052/
